In [ ]:
pip install transformers datasets torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/My Drive/CS 7650/preprocessed_us_bills/

/content/drive/My Drive/CS 7650/preprocessed_us_bills


In [ ]:
from datasets import load_from_disk
from datasets import Dataset

# Load the entire dataset directory
dataset = load_from_disk('/content/drive/MyDrive/CS 7650/preprocessed_us_bills/train')
train_dataset = dataset
train_dataset = Dataset.from_file('/content/drive/MyDrive/CS 7650/preprocessed_us_bills/train/data-00000-of-00002.arrow')

dataset = load_from_disk('/content/drive/MyDrive/CS 7650/preprocessed_us_bills/validation')
test_dataset = dataset
test_dataset = Dataset.from_file('/content/drive/MyDrive/CS 7650/preprocessed_us_bills/validation/data-00000-of-00001.arrow')

In [ ]:
import torch

from transformers import BertTokenizer, BertForMaskedLM, Trainer, TrainingArguments
from datasets import Dataset

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenization function with masking
def tokenize_and_mask(examples):
    # Tokenize
    inputs = tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128, return_tensors="pt")
    inputs['labels'] = inputs.input_ids.detach().clone()

    # Create random mask
    rand = torch.rand(inputs.input_ids.shape)
    mask_arr = (rand < 0.15) * (inputs.input_ids != tokenizer.cls_token_id) * \
               (inputs.input_ids != tokenizer.sep_token_id) * (inputs.input_ids != tokenizer.pad_token_id)

    selection = []

    for i in range(inputs.input_ids.shape[0]):
        selection.append(torch.flatten(mask_arr[i].nonzero()).tolist())

    for i in range(inputs.input_ids.shape[0]):
        inputs.input_ids[i, selection[i]] = tokenizer.mask_token_id

    return inputs

#random.seed(42)
random_indices = random.sample(range(len(train_dataset)), 1000)
train_dataset_1000 = train_dataset.select(random_indices)
#test_dataset_200 = test_dataset.select(range(200))

# Assuming 'train_dataset' is already loaded and contains your data
texts = train_dataset_1000['text']
dataset = Dataset.from_dict({'text': texts})

# Apply tokenization and masking
tokenized_datasets = dataset.map(tokenize_and_mask, batched=True)

# Load pre-trained BERT model for MLM
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
random_indices = random.sample(range(len(test_dataset)), 200)
test_dataset_200 = test_dataset.select(random_indices)
test_texts = test_dataset_200['text']
test_data = Dataset.from_dict({'text': test_texts})

# Apply tokenization and masking
tokenized_test = test_data.map(tokenize_and_mask, batched=True)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
# tokenized_datasets_1000 = tokenized_datasets.select(range(1000))
# tokenized_test_1000 = tokenized_test.select(range(1000))

In [ ]:
import numpy as np

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    report_to="none",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,  # Training batch size
    per_device_eval_batch_size=8,   # Reduced evaluation batch size
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    gradient_accumulation_steps=2,  # Simulate larger batch size
    gradient_checkpointing=True,
)


from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    num_classes = tokenizer.vocab_size # Assuming tokenizer has vocab_size property
    logits = logits.reshape(logits.shape[0], logits.shape[1], num_classes)
    predictions = np.argmax(logits, axis=-1)
    mask = labels != -100  # Ignore masked tokens in the labels
    predictions = predictions[mask]
    labels = labels[mask]
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    decoded_labels = [tokenizer.decode([token_id], skip_special_tokens=True) for token_id in labels]
    decoded_predictions = [tokenizer.decode([token_id], skip_special_tokens=True) for token_id in predictions]
    print("Sample decoded labels:", decoded_labels)
    print("Sample decoded predictions:", decoded_predictions)
    return {"accuracy": acc, "f1": f1}


# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Fine-tune the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,F1
0,No log,0.669979,0.909609,0.911057
2,No log,0.524790,0.923047,0.922411


Sample decoded labels: ['', '116', 's', '##29', '##0', 'native', 'youth', 'tribal', 'officer', 'protection', 'act', 'senate', 'en', 'pursuant', 'title', '17', 'section', '105', 'united', 'state', 'code', 'file', 'subject', 'copyright', 'protection', 'public', 'domain', 'ii', '##11', '##6th', 'congress', '##1', '##st', 'session', '290', '##in', 'senate', 'united', 'states', '##jan', '##uary', '31', '2019', '##m', '##r', 'ud', '##all', 'mu', '##rk', '##owski', 'smith', 'introduced', 'following', 'bill', 'read', 'twice', 'referred', 'committee', 'indian', 'affairs', '##a', 'bill', '##to', 'protect', 'native', 'child', 'promote', 'public', 'safety', 'indian', 'country', 'title', '##thi', '##s', 'act', 'may', 'cited', 'native', 'youth', 'tribal', 'officer', 'protection', 'act', 'find', 'following', '1', 'american', 'indian', 'alaska', 'native', 'time', 'likely', 'experience', 'violent', 'crime', 'least', '2', 'time', 'likely', 'experience', 'rape', 'sexual', 'assault', 'crime', '2', 'vast',

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Sample decoded labels: ['', '116', 's', '##29', '##0', 'native', 'youth', 'tribal', 'officer', 'protection', 'act', 'senate', 'en', 'pursuant', 'title', '17', 'section', '105', 'united', 'state', 'code', 'file', 'subject', 'copyright', 'protection', 'public', 'domain', 'ii', '##11', '##6th', 'congress', '##1', '##st', 'session', '290', '##in', 'senate', 'united', 'states', '##jan', '##uary', '31', '2019', '##m', '##r', 'ud', '##all', 'mu', '##rk', '##owski', 'smith', 'introduced', 'following', 'bill', 'read', 'twice', 'referred', 'committee', 'indian', 'affairs', '##a', 'bill', '##to', 'protect', 'native', 'child', 'promote', 'public', 'safety', 'indian', 'country', 'title', '##thi', '##s', 'act', 'may', 'cited', 'native', 'youth', 'tribal', 'officer', 'protection', 'act', 'find', 'following', '1', 'american', 'indian', 'alaska', 'native', 'time', 'likely', 'experience', 'violent', 'crime', 'least', '2', 'time', 'likely', 'experience', 'rape', 'sexual', 'assault', 'crime', '2', 'vast',

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Sample decoded labels: ['', '116', 's', '##29', '##0', 'native', 'youth', 'tribal', 'officer', 'protection', 'act', 'senate', 'en', 'pursuant', 'title', '17', 'section', '105', 'united', 'state', 'code', 'file', 'subject', 'copyright', 'protection', 'public', 'domain', 'ii', '##11', '##6th', 'congress', '##1', '##st', 'session', '290', '##in', 'senate', 'united', 'states', '##jan', '##uary', '31', '2019', '##m', '##r', 'ud', '##all', 'mu', '##rk', '##owski', 'smith', 'introduced', 'following', 'bill', 'read', 'twice', 'referred', 'committee', 'indian', 'affairs', '##a', 'bill', '##to', 'protect', 'native', 'child', 'promote', 'public', 'safety', 'indian', 'country', 'title', '##thi', '##s', 'act', 'may', 'cited', 'native', 'youth', 'tribal', 'officer', 'protection', 'act', 'find', 'following', '1', 'american', 'indian', 'alaska', 'native', 'time', 'likely', 'experience', 'violent', 'crime', 'least', '2', 'time', 'likely', 'experience', 'rape', 'sexual', 'assault', 'crime', '2', 'vast',

TrainOutput(global_step=93, training_loss=0.7717372114940356, metrics={'train_runtime': 132.9785, 'train_samples_per_second': 22.56, 'train_steps_per_second': 0.699, 'total_flos': 194771561472000.0, 'train_loss': 0.7717372114940356, 'epoch': 2.9523809523809526})

In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 0.5340056419372559, 'eval_accuracy': 0.92390625, 'eval_f1': 0.9231711261185163, 'eval_runtime': 4.7743, 'eval_samples_per_second': 41.891, 'eval_steps_per_second': 5.236, 'epoch': 2.9523809523809526}


In [ ]:
trainer.compute_metrics(labels, predictions)

NameError: name 'labels' is not defined

In [ ]:
#BERT without finetuning
from transformers import BertTokenizer, BertForMaskedLM

# Load pre-trained model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a Bert

In [ ]:
import random
from transformers import BertTokenizer

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Example sentences for tokenization
sentences = test_texts

# Function to randomly mask 15% of the tokens
def mask_tokens(sentences):
    masked_sentences = []
    expected_completions = {}

    for sentence in sentences:
        tokens = tokenizer.tokenize(sentence)
        token_ids = tokenizer.convert_tokens_to_ids(tokens)

        # Determine number of tokens to mask
        mask_count = max(1, int(0.15 * len(tokens)))  # At least one token

        # Randomly select tokens to mask
        mask_indices = random.sample(range(len(tokens)), mask_count)

        # Capture expected completions
        expected_completions[sentence] = [tokens[idx] for idx in mask_indices]

        # Mask selected tokens
        for idx in mask_indices:
            tokens[idx] = '[MASK]'

        masked_sentence = tokenizer.convert_tokens_to_string(tokens)
        masked_sentences.append(masked_sentence)

    return masked_sentences, expected_completions

# Apply masking and capture expected completions
masked_sentences, expected_completions = mask_tokens(sentences)

# Tokenize masked sentences
device = torch.device('cpu')
inputs = tokenizer(masked_sentences, return_tensors='pt', padding=True, truncation=True).to(device)

# Get model predictions
model.eval()
predictions = []
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

    # Extract predicted token IDs for masked positions
    for i, sentence in enumerate(masked_sentences):
        input_ids = inputs['input_ids'][i]
        mask_token_index = (input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[0]

        predicted_token_id = logits[i, mask_token_index].argmax(axis=-1).item()
        predicted_token = tokenizer.decode([predicted_token_id])

        predictions.append(predicted_token)

# Calculate accuracy
correct_predictions = 0
total_predictions = 0

for original_sentence, expected_tokens in expected_completions.items():
    predicted_tokens_for_sentence = predictions[:len(expected_tokens)]
    predictions = predictions[len(expected_tokens):]  # Shift predictions for next sentence

    correct_predictions += sum(p == e for p, e in zip(predicted_tokens_for_sentence, expected_tokens))
    total_predictions += len(expected_tokens)

accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
print(f"Accuracy: {accuracy:.2%}")

RuntimeError: a Tensor with 77 elements cannot be converted to Scalar

In [ ]:
!pip install datasets --upgrade

In [ ]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import evaluate

# Load the accuracy metric
metric = evaluate.load("accuracy")

# Function to predict the next word and update the metric
device = torch.device('cpu')
model.to(device)
def predict_next_word(examples):
    # Tokenize the input text
    inputs = tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128, return_tensors="pt").to(device)

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted token IDs
    predicted_token_ids = torch.argmax(outputs.logits, dim=-1).to(device)

    # Remove padding tokens from labels and predictions
    labels = inputs.input_ids
    labels_no_pad = [label[label != tokenizer.pad_token_id] for label in labels]
    predictions_no_pad = [prediction[label != tokenizer.pad_token_id] for prediction, label in zip(predicted_token_ids, labels)]

    # Update the accuracy metric
    metric.add_batch(predictions=predictions_no_pad, references=labels_no_pad)

    return {}

# Apply the prediction function to the test dataset
test_data.map(predict_next_word, batched=True).to(device)

# Calculate and print the accuracy
accuracy = metric.compute()['accuracy']
print(f"Next Word Prediction Accuracy: {accuracy}")

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

ValueError: Predictions and/or references don't match the expected format.
Expected format: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)},
Input predictions: [tensor([  101, 12904,  1055, 24594,  2692,  3128,  3360,  8807,  2961,  3860,
         2552,  4001,  4372, 27081,  2516,  2459,  2930,  8746,  2142,  2110,
         3642,  5371,  3395,  9385,  3860,  2270,  5884,  2462, 14526, 25994,
         3519,  2487,  3367,  5219, 17222,  2378,  4001,  2142,  2163,  8405,
        24384,  2861, 10476,  2213,  2099, 20904,  8095, 14163,  8024, 15249,
         3044,  3107,  2206,  3021,  3191,  3807,  3615,  2837,  2796,  3821,
         2050,  3021,  3406,  4047,  3128,  2775,  5326,  2270,  3808,  2796,
         2406,  2516, 15222,  2015,  2552,  2089,  6563,  3128,  3360,  8807,
         2961,  3860,  2552,  2424,  2206,  1015,  2137,  2796,  7397,  3128,
         2051,  3497,  3325,  6355,  4126,  2560,  1016,  2051,  3497,  3325,
         9040, 

In [ ]:
import numpy as np
import evaluate

# Load the accuracy metric
metric = evaluate.load("accuracy")

# Function to predict the next word and update the metric
device = torch.device('cpu')
model.to(device)
def predict_next_word(examples):
    # Tokenize the input text
    inputs = tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128, return_tensors="pt").to(device)

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted token IDs
    predicted_token_ids = torch.argmax(outputs.logits, dim=-1).to(device)

    # Remove padding tokens from labels and predictions and flatten to lists
    labels = inputs.input_ids.cpu().numpy().tolist() # Convert to list of lists
    predictions = predicted_token_ids.cpu().numpy().tolist() # Convert to list of lists

    # Convert list of lists into a single list of tokens by removing pad token and flattening each sub-list

    labels_no_pad = [token for sublist in labels for token in sublist if token != tokenizer.pad_token_id]
    predictions_no_pad = [token for sublist in predictions for token in sublist if token != tokenizer.pad_token_id]

    # Update the accuracy metric
    metric.add_batch(predictions=predictions_no_pad, references=labels_no_pad)

    return {}

# Apply the prediction function to the test dataset
test_data.map(predict_next_word, batched=True)

# Calculate and print the accuracy
accuracy = metric.compute()['accuracy']
print(f"Next Word Prediction Accuracy: {accuracy}")

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Next Word Prediction Accuracy: 0.9998029789581527


In [ ]:
import numpy as np
import evaluate
import torch

# Load the accuracy metric
metric = evaluate.load("accuracy")

# Function to predict the masked words and update the metric
device = torch.device('cpu')
model.to(device)

def predict_masked_words(examples):
    # Tokenize the input text
    inputs = tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128, return_tensors="pt").to(device)

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted token IDs
    predicted_token_ids = torch.argmax(outputs.logits, dim=-1)

    # Get the mask token indices
    mask_token_indices = torch.where(inputs.input_ids == tokenizer.mask_token_id)

    # Extract predictions and labels only for masked positions
    predictions = predicted_token_ids[mask_token_indices].cpu().numpy().tolist()
    labels = inputs.input_ids[mask_token_indices].cpu().numpy().tolist()

    # Debugging: Check if predictions and labels are empty
    print("Predictions:", predictions)
    print("Labels:", labels)
    print("Lengths:", len(predictions), len(labels))

    if not predictions or not labels:
        print("Warning: Empty predictions or labels for this batch.")
        return {}  # Skip this batch if empty

    # Update the accuracy metric
    metric.add_batch(predictions=predictions, references=labels)

    return {}

# Apply the prediction function to the test dataset
tokenized_test.map(predict_masked_words, batched=True)

# Calculate and print the accuracy
accuracy = metric.compute()['accuracy']
print(f"Masked Language Modeling Accuracy: {accuracy}")

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Predictions: []
Labels: []
Lengths: 0 0


TypeError: 'in <string>' requires string as left operand, not int